In [147]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split

In [148]:
data = pd.read_csv('tumor.csv')

In [149]:
data.drop(['Sample code number'], axis=1, inplace = True)
# Lets trim the data

In [150]:
data.Class = [1 if each == 4 else 0 for each in data.Class] 
# This is for easier interpretation of data... 4 represented a malignant tumor while 2 a benign tumor, I changed the values to 1 and 0 respectively

In [151]:
y = data.Class.values
x = data.drop(['Class'], axis=1)
#Segregating the database characters and result

In [152]:
x = (x - np.min(x)) / (np.max(x) - np.min(x))

C:\Python310\lib\site-packages\numpy\core\fromnumeric.py:84: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)
C:\Python310\lib\site-packages\numpy\core\fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


In [153]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
# I would use 20% of the dataset for test purpose and 80% as the training set.

x_train = x_train.T
y_train = y_train.T
x_test = x_test.T
y_test = y_test.T

In [154]:
def initialize_weight_bias(dimension):
    w = np.full((dimension,1), 0.01) 
    b = 0.0
    return w,b

In [155]:
def sigmoid(z):
    y_head = 1 / (1 + np.exp(-z))
    return y_head

In [156]:
def forward_backward_propagation(w, b, x_train, y_train):
   
    z = np.dot(w.T, x_train) + b
    y_head = sigmoid(z)
    
    loss = -(1 - y_train) * np.log(1 - y_head) - y_train * np.log(y_head)     
    cost = (np.sum(loss)) / x_train.shape[1]                               
    
   
    derivative_weight = (np.dot(x_train,((y_head-y_train).T)))/x_train.shape[1]
    derivative_bias = np.sum(y_head-y_train)/x_train.shape[1]
    
    gradients = {'derivative_weight': derivative_weight, 'derivative_bias': derivative_bias}
    
    return cost, gradients

In [157]:
def update(w, b, x_train, y_train, learning_rate, iteration):
    cost_list = []
      
    for i in range(iteration):
        cost, gradients = forward_backward_propagation(w, b, x_train, y_train)
        cost_list.append(cost)
    
        # Updating weight and bias values:
        w = w - learning_rate * gradients['derivative_weight']
        b = b - learning_rate * gradients['derivative_bias']
    
    parameters = {'weight': w, 'bias':b}
    
    return parameters, gradients, cost_list

In [158]:
def prediction(w, b, x_test):
    z = sigmoid(np.dot(w.T, x_test) + b)
    y_prediction = np.zeros((1,x_test.shape[1]))
    
    for i in range(z.shape[1]):
        if z[0,i]<= 0.5:
            y_prediction[0,i] = 0
        else:
            y_prediction[0,i] = 1
            
    return y_prediction

In [159]:
def logistic_regression(x_train, y_train, x_test, y_test, learning_rate, iteration):
    dimension = x_train.shape[0]
    w, b = initialize_weight_bias(dimension)   
    
    parameters, gradients, cost_list = update(w, b, x_train, y_train, learning_rate, iteration)
    
    # Lets use x_test for predicting y:
    y_test_predictions = prediction(parameters['weight'], parameters['bias'], x_test) 
    
    print('Test accuracy: {}%'.format(100 - np.mean(np.abs(y_test_predictions - y_test))*100))

In [160]:
logistic_regression(x_train, y_train, x_test, y_test, learning_rate=1, iteration=400)

Test accuracy: 97.08029197080292%
